This notebook will demonstrate how to simulate images for the Rubin Auxiliary Telescope (aka AuxTel or AT).

Our simulations will depend heavily on two packages:
- `galsim` which is an astronomical image simulation toolkit
- `batoid` which is a raytracing package that we use to simuate the optics of the telescope

We will also use `wfsim`, which is a little wrapper package that connects elements of `galsim` and `batoid`, making it easy to quickly simulate images for the AOS.
Essentially, `galsim` is used to simulate the atmosphere and sensors, while `batoid` is used to simulate the optics of the telescope (i.e. the mirrors, lenses, etc.).

Let's import these packages (plus `numpy` and `matplotlib`):

In [1]:
import numpy as np # standard package for doing calculations
import matplotlib.pyplot as plt # standard package for plotting

import batoid
import galsim
import wfsim

First, let's create a `batoid` simulator for the unperturbed AuxTel:

In [2]:
auxtel0 = batoid.Optic.fromYaml("AuxTel.yaml")

Let's also load the bandpass we will be observing through.
We will assume the LSST r band.

In [3]:
bandpass = galsim.Bandpass("LSST_r.dat", wave_type='nm')

Now let's simulate an in-focus star at the center of the focal plane.
To do that, we need to set observations and atmospheric parameters.
Read the comments below to get an idea for what each parameter means.

In [4]:
# parameters associated with the observing conditions
obs_params ={
    # zenith=0 means straight up, while zenith=90 is the horizon
    # greater zenith angle means more atmospheric turbulence
    "zenith": 30 * galsim.degrees,

    # raw_seeing sets how much atmospheric turbulence there is at zenith=0
    # wfsim uses this to calculate how much turbulence there is at 
    # the zenith angle given above
    "raw_seeing": 0.7 * galsim.arcsec,

    # these next three parameters set attributes of the atmosphere, which
    # help determine how much turbulence there is
    "temperature": 293, # Kelvin
    "pressure": 69, # kPa
    "H2O_pressure": 1, # kPa

    # wavelength of the observed light. this is important because light of 
    # longer/shorter wavelength refracts less/more in the atmosphere
    "wavelength": bandpass.effective_wavelength, # nanometers

    # the AuxTel exposure time
    "exptime": 30, # seconds
}

# parameters associated with computational aspects of simulating the atmosphere
atm_params = {
    "screen_size": 819.2,
    "screen_scale": 0.1,
    "nproc": 6,
}

Now we can create a `SimpleSimulator` with `wfsim`.
This is the object we will use to simulate images.
Because we will be creating many versions of `SimpleSimulator` in this notebook, I will write a function that creates `SimpleSimulator`'s with our set up.

In [5]:
def create_simulator(telescope: batoid.Optic) -> wfsim.SimpleSimulator:
    """Create a SimpleSimulator built on the provided telescope.

    Uses the obs_params and atm_params defined earlier in this notebook.

    Parameters
    ----------
    telescope: batoid.Optic
        A batoid.Optic that describes the telescope.

    Returns
    -------
    wfsim.SimpleSimulator
        A SimpleSimulator built on the provided telescope, 
        using the previously defined settings.
    """
    # create a random number generator
    rng = np.random.default_rng(0)

    # create the SimpleSimulator
    simulator = wfsim.SimpleSimulator(
        obs_params,
        atm_params,
        telescope,
        bandpass,
        shape=(512, 512),
        rng=rng,
    )

    return simulator

In [6]:
infocus_simulator = create_simulator(auxtel0)

: 

: 

#### To do: Simulate a star, with a position, temperature, flux, then add a background

#### To do: Plot the star. Talk about how it's small because it is in focus.

#### To do: push the telescope out of focus, and plot simulate the star again.

#### To do: now perturb the telescope, and simulate again

#### To do: wrap up and tell Dominik to play around with different perturbation sizes

For these simulations, we have only concerned ourselves with optical misalignment.
However, for the larger Rubin Telescope, we also need to worry about the bending of the mirrors.
[This notebook](https://colab.research.google.com/drive/1YMh0B_hcBSm9cq02aQlyDHdGJv5fjmcV#scrollTo=NGznLCN7bHOg) on Google Colab provides an interactive demonstration of how the bending impacts the donut images.

If you run every cell (this takes a minute or two) and scroll down to the bottom, you will see plots of the "wavefront", and the intra- and extra-focal donut images.
Above these are fields where you can change the position of the donuts, as well as the mirror bending (Z4-Z34).
Try playing around with these to see how the donuts change.

This won't matter for what we are doing with AuxTel, but it is cool to see.